In [ ]:
import re
from typing import List, Union
# Python内置模块，用于格式化和包装文本
import textwrap
import time

from langchain.agents import (
    Tool,          # 可用工具
    AgentExecutor, # Agent执行
    LLMSingleActionAgent, # 定义Agent
    AgentOutputParser, # 输出结果解析
)
from langchain.prompts import StringPromptTemplate
# LLMChain，包含一个PromptTemplate和一个LLM
from langchain_community.llms import Tongyi  # 导入通义千问Tongyi模型
from langchain import LLMChain
# Agent执行，Agent结束
from langchain.schema import AgentAction, AgentFinish
# PromptTemplate: 管理LLMs的Prompts
from langchain.prompts import PromptTemplate
from langchain.llms.base import BaseLLM

# 定义了LLM的Prompt Template
CONTEXT_QA_TMPL = """
根据以下提供的信息，回答用户的问题
信息：{context}

问题：{query}
"""
CONTEXT_QA_PROMPT = PromptTemplate(
    input_variables=["query", "context"],
    template=CONTEXT_QA_TMPL,
)

# 输出结果显示，每行最多60字符，每个字符显示停留0.1秒（动态显示效果）
def output_response(response: str) -> None:
    if not response:
        exit(0)
    # 每行最多60个字符
    for line in textwrap.wrap(response, width=60):
        for word in line.split():
            for char in word:
                print(char, end="", flush=True)
                time.sleep(0.1)  # Add a delay of 0.1 seconds between each character
            print(" ", end="", flush=True)  # Add a space between each word
        print()  # Move to the next line after each line is printed
    # 遇到这里，这个问题的回答就结束了
    print("----------------------------------------------------------------")

# 模拟公司产品和公司介绍的数据源
class TeslaDataSource:
    def __init__(self, llm: BaseLLM):
        self.llm = llm

    # 工具1：产品描述
    def find_product_description(self, product_name: str) -> str:
        """模拟公司产品的数据库"""
        product_info = {
            "Model 3": "具有简洁、动感的外观设计，流线型车身和现代化前脸。定价23.19-33.19万",
            "Model Y": "在外观上与Model 3相似，但采用了更高的车身和更大的后备箱空间。定价26.39-36.39万",
            "Model X": "拥有独特的翅子门设计和更加大胆的外观风格。定价89.89-105.89万",
        }
        # 基于产品名称 => 产品描述
        return product_info.get(product_name, "没有找到这个产品")

    # 工具2：公司介绍
    def find_company_info(self, query: str) -> str:
        """模拟公司介绍文档数据库，让llm根据信息回答问题"""
        context = """
        特斯拉最知名的产品是电动汽车，其中包括Model S、Model 3、Model X和Model Y等多款车型。
        特斯拉以其技术创新、高性能和领先的自动驾驶技术而闻名。公司不断推动自动驾驶技术的研发，并在车辆中引入了各种驾驶辅助功能，如自动紧急制动、自适应巡航控制和车道保持辅助等。
        """
        # prompt模板 = 上下文context + 用户的query
        prompt = CONTEXT_QA_PROMPT.format(query=query, context=context)
        # 使用LLM进行推理
        return self.llm(prompt)


AGENT_TMPL = """按照给定的格式回答以下问题。你可以使用下面这些工具：

{tools}

回答时需要遵循以下用---括起来的格式：

---
Question: 我需要回答的问题
Thought: 回答这个上述我需要做些什么
Action: "{tool_names}" 中的一个工具名
Action Input: 选择这个工具所需要的输入
Observation: 选择这个工具返回的结果
...（这个 思考/行动/行动输入/观察 可以重复N次）
Thought: 我现在知道最终答案
Final Answer: 原始输入问题的最终答案
---

现在开始回答，记得在给出最终答案前，需要按照指定格式进行一步一步的推理。

Question: {input}
{agent_scratchpad}
"""


class CustomPromptTemplate(StringPromptTemplate):
    template: str  # 标准模板
    tools: List[Tool]  # 可使用工具集合

    def format(self, **kwargs) -> str:
        """
        按照定义的 template，将需要的值都填写进去。
        Returns:
            str: 填充好后的 template。
        """
        # 取出中间步骤并进行执行
        intermediate_steps = kwargs.pop("intermediate_steps")  
        print('intermediate_steps=', intermediate_steps)
        print('='*30)
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # 记录下当前想法 => 赋值给agent_scratchpad
        kwargs["agent_scratchpad"] = thoughts  
        # 枚举所有可使用的工具名+工具描述
        kwargs["tools"] = "\n".join(
            [f"{tool.name}: {tool.description}" for tool in self.tools]
        )  
        # 枚举所有的工具名称
        kwargs["tool_names"] = ", ".join(
            [tool.name for tool in self.tools]
        )  
        cur_prompt = self.template.format(**kwargs)
        #print(cur_prompt)
        return cur_prompt

"""
    对Agent返回结果进行解析，有两种可能：
    1）还在思考中 AgentAction
    2）找到了答案 AgentFinal
"""
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        """
        解析 llm 的输出，根据输出文本找到需要执行的决策。
        Args:
            llm_output (str): _description_
        Raises:
            ValueError: _description_
        Returns:
            Union[AgentAction, AgentFinish]: _description_
        """
        # 如果句子中包含 Final Answer 则代表已经完成
        if "Final Answer:" in llm_output:  
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )

        # 需要进行 AgentAction
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"  # 解析 action_input 和 action
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Agent执行
        return AgentAction(
            tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
        )

# 设置通义千问API密钥
DASHSCOPE_API_KEY = 'sk-882e296067b744289acf27e6e20f3ec0'

if __name__ == "__main__":
    # 定义LLM
    llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY)  # 使用通义千问qwen-turbo模型
    # 自有数据
    tesla_data_source = TeslaDataSource(llm)
    # 定义的Tools
    tools = [
        Tool(
            name="查询产品名称",
            func=tesla_data_source.find_product_description,
            description="通过产品名称找到产品描述时用的工具，输入的是产品名称",
        ),
        Tool(
            name="公司相关信息",
            func=tesla_data_source.find_company_info,
            description="当用户询问公司相关的问题，可以通过这个工具了解公司信息",
        ),
    ]
    
    # 用户定义的模板
    agent_prompt = CustomPromptTemplate(
        template=AGENT_TMPL,
        tools=tools,
        input_variables=["input", "intermediate_steps"],
    )
    # Agent返回结果解析
    output_parser = CustomOutputParser()
    # 最常用的Chain, 由LLM + PromptTemplate组成
    llm_chain = LLMChain(llm=llm, prompt=agent_prompt)
    # 定义的工具名称
    tool_names = [tool.name for tool in tools]
    # 定义Agent = llm_chain + output_parser + tools_names
    agent = LLMSingleActionAgent(
        llm_chain=llm_chain,
        output_parser=output_parser,
        stop=["\nObservation:"],
        allowed_tools=tool_names,
    )
    # 定义Agent执行器 = Agent + Tools
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent, tools=tools, verbose=True
    )

    # 主过程：可以一直提问下去，直到Ctrl+C
    while True:
        try:
            user_input = input("请输入您的问题：")
            response = agent_executor.run(user_input)
            output_response(response)
        except KeyboardInterrupt:
            break

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18896\4265487943.py:201: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=agent_prompt)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_18896\4265487943.py:205: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/cre

请输入您的问题： model 3怎么样


C:\Users\Administrator\AppData\Local\Temp\ipykernel_18896\4265487943.py:220: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run(user_input)




> Entering new AgentExecutor chain...
intermediate_steps= []
---
Question: model 3怎么样  
Thought: 用户想了解Model 3的情况，我需要查询这款产品的详细信息。  
Action: 查询产品名称  
Action Input: Model 3  

Observation:具有简洁、动感的外观设计，流线型车身和现代化前脸。定价23.19-33.19万
intermediate_steps= [(AgentAction(tool='查询产品名称', tool_input='Model 3', log='---\nQuestion: model 3怎么样  \nThought: 用户想了解Model 3的情况，我需要查询这款产品的详细信息。  \nAction: 查询产品名称  \nAction Input: Model 3  '), '具有简洁、动感的外观设计，流线型车身和现代化前脸。定价23.19-33.19万')]
我现在知道最终答案  
Final Answer: Model 3是一款具有简洁、动感外观设计的车型，其流线型车身和现代化前脸非常吸引人。价格区间为23.19万至33.19万。

> Finished chain.
Model 
3是一款具有简洁、动感外观设计的车型，其流线型车身和现代化前脸非常吸引人。价格区间为23.19万至33.19万。 
----------------------------------------------------------------


请输入您的问题： 特斯拉有哪些产品




> Entering new AgentExecutor chain...
intermediate_steps= []
---
Question: 特斯拉有哪些产品  
Thought: 要回答这个问题，我需要查询特斯拉公司所生产的产品名称。  
Action: 查询产品名称  
Action Input: 特斯拉  

Observation:没有找到这个产品
intermediate_steps= [(AgentAction(tool='查询产品名称', tool_input='特斯拉', log='---\nQuestion: 特斯拉有哪些产品  \nThought: 要回答这个问题，我需要查询特斯拉公司所生产的产品名称。  \nAction: 查询产品名称  \nAction Input: 特斯拉  '), '没有找到这个产品')]
查询工具未能直接提供特斯拉的产品列表，但我可以通过已知信息或公司相关信息来获取特斯拉的主要产品。

---
Thought: 我需要使用公司相关信息工具来了解特斯拉的业务范围和主要产品。  
Action: 公司相关信息  
Action Input: 特斯拉  

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18896\4265487943.py:76: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.llm(prompt)




Observation:特斯拉是一家全球知名的电动汽车及能源公司，以其创新的技术和高性能产品而闻名。其主要产品包括多款电动车型，如 **Model S**、**Model 3**、**Model X** 和 **Model Y**。这些车辆不仅以环保的电力驱动为核心，还配备了先进的技术功能。

特斯拉在自动驾驶领域处于领先地位，不断推动相关技术研发。其车辆内置了多种驾驶辅助功能，例如：
- **自动紧急制动**：在检测到潜在碰撞时自动刹车。
- **自适应巡航控制**：根据前方车辆的速度自动调整车速。
- **车道保持辅助**：帮助车辆保持在车道内行驶。

通过这些技术创新，特斯拉不仅改变了汽车行业，还为可持续能源的未来提供了重要解决方案。如果您有具体问题或想了解更多细节，请告诉我！
intermediate_steps= [(AgentAction(tool='查询产品名称', tool_input='特斯拉', log='---\nQuestion: 特斯拉有哪些产品  \nThought: 要回答这个问题，我需要查询特斯拉公司所生产的产品名称。  \nAction: 查询产品名称  \nAction Input: 特斯拉  '), '没有找到这个产品'), (AgentAction(tool='公司相关信息', tool_input='特斯拉', log='查询工具未能直接提供特斯拉的产品列表，但我可以通过已知信息或公司相关信息来获取特斯拉的主要产品。\n\n---\nThought: 我需要使用公司相关信息工具来了解特斯拉的业务范围和主要产品。  \nAction: 公司相关信息  \nAction Input: 特斯拉  '), '特斯拉是一家全球知名的电动汽车及能源公司，以其创新的技术和高性能产品而闻名。其主要产品包括多款电动车型，如 **Model S**、**Model 3**、**Model X** 和 **Model Y**。这些车辆不仅以环保的电力驱动为核心，还配备了先进的技术功能。\n\n特斯拉在自动驾驶领域处于领先地位，不断推动相关技术研发。其车辆内置了多种驾驶辅助功能，例如：\n- **自动紧急制动**：在检测到潜在碰撞时自动刹车。\n- **自适应巡航控制**：根据前方车辆的速度自动调整车速。\n- **车道保持辅助**：帮助车辆保

In [3]:
import os
#DASHSCOPE_API_KEY = 'sk-882e296067b744289acf27e6e20f3ec0'
os.getenv("DASHSCOPE_API_KEY")

'sk-dd7ae33a0056483a82660b9392f4eedc'